In [1]:
source("~/smartas/pipeline/scripts/variablesAndFunctions.r")

# read switch information
switches <- read.delim("../pancancer/candidateList_info.agg.tsv",row.names=NULL)

# read mutual exclusion info
wgs <- read.delim("../pancancer/mutations/gene_wgs_mutations_all_switches.txt",row.names=NULL)
wgs <- merge(wgs,switches)

In [10]:
wgs[wgs$p.o < 0.05,]

,GeneId,Symbol,Normal_transcript,Tumor_transcript,MS,M,S,N,p.o,Normal_protein,Tumor_protein,Annotation,DriverAnnotation,NotNoise,IsModel,IsFunctional,Driver,Druggable,CDS_Normal,CDS_Tumor,CDS_change,UTR_change,Tumors,PatientNumber,Percentage,Patients_affected
111,10196,PRMT3,uc001mqb.2,uc001mqc.2,3,3,1,21,0.02222222,O60678,None,HALLMARK_MYC_TARGETS_V2,d1,1,1,1,0,0,1,1,1,1,"brca,hnsc,kirc,kirp",20,0.004502476,"A50YT,A5Q5T,A6IJT,4900T,5168T,3444T,5905T,5682T,5199T,5566T,4143T,5590T,5844T,4639T,5696T,4852T,3365T,4355T,7287T,A5NLT"
166,10322,SMYD5,uc002siw.2,uc002six.1,1,2,0,92,0.03157895,Q6GMV2,None,Nothing,Nothing,1,1,1,0,0,1,0,1,1,"brca,coad,hnsc,kich,kirp,lihc,lusc,prad",19,0.004277353,"A5YLT,A0CRT,A03NT,4743T,A4AQT,6945T,A6IIT,A6EKT,A6DAT,A6JTT,7269T,5973T,8409T,A656T,7963T,A119T,2720T,A59VT,A67OT"
294,10672,GNA13,uc002jfc.2,uc010wqh.1,3,10,0,61,0.004411946,Q14344,None,SIGNAL_TRANSDUCTION,driver,1,1,1,1,0,1,1,1,1,"coad,hnsc,kich,kirc,kirp,lihc,luad,lusc,prad,thca",173,0.03894642,"5860T,4315T,7870T,A4ZBT,6225T,A6H7T,A49CT,A461T,6872T,A620T,A45YT,7269T,A4CIT,A61OT,A634T,5152T,A61ST,A45TT,A5Q6T,6945T,A6EMT,A6I9T,A6ENT,A50JT,4074T,A6DJT,6436T,A642T,7871T,A5Q5T,A6JDT,5333T,5966T,A6JMT,A468T,A6JTT,A4CAT,A61VT,6870T,A6DAT,A66ST,A6DDT,6869T,A6DBT,A6DGT,8405T,8403T,8404T,8415T,4714T,3451T,4876T,A564T,4697T,4713T,5092T,4707T,4969T,4817T,4813T,A4VZT,5084T,5400T,5185T,5100T,4819T,4696T,4718T,4833T,4776T,3426T,4875T,4706T,4852T,A4VXT,5700T,5170T,4700T,4900T,4810T,4828T,4159T,4335T,4691T,3455T,4873T,4337T,4878T,5676T,5677T,5469T,5191T,4887T,4824T,4988T,3434T,4153T,4165T,6090T,4839T,4992T,4842T,4167T,5678T,3427T,4891T,4981T,4968T,3436T,4849T,4843T,4892T,4912T,5468T,4841T,4847T,A654T,A898T,A7VFT,5881T,A5EAT,A562T,A83TT,A6M8T,A6M9T,5886T,A7SMT,A59TT,A856T,3471T,A40XT,A560T,A47NT,7502T,A5NIT,5560T,3472T,4114T,7173T,A55ZT,A6GET,A47MT,A5QZT,A44BT,A6HPT,7732T,A119T,1676T,7815T,6597T,1677T,4507T,A443T,5044T,A5B5T,7033T,A512T,A4CLT,4566T,5027T,4130T,5819T,2726T,A5HOT,3421T,7465T,A5ENT,4589T,3783T,5784T,A59VT,6373T,A3PUT"
728,123811,C16orf63,uc002dec.1,uc002ded.1,1,3,0,91,0.04210526,Q96NB1,None,Nothing,Nothing,1,1,1,0,0,1,1,1,0,"brca,hnsc,kich,kirc,kirp,luad,prad,thca",53,0.01193156,"A3X8T,A54OT,A7HQT,A04PT,A572T,A3QQT,A574T,A66LT,A56DT,A6FQT,A576T,A5S0T,A50YT,A3XVT,A6FRT,A3TMT,A5FKT,A6JTT,A6EPT,A6IJT,8327T,A564T,A6GET,A6M9T,A6M8T,A71RT,A55ZT,3472T,A654T,A5NIT,A5EAT,A7VFT,A47NT,A492T,A6ILT,A67OT,A723T,A6ALT,A59VT,A6AQT,A6INT,A67NT,A6E3T,A6IMT,A46FT,A67QT,7325T,A6ANT,A6IOT,A7AYT,A6AST,A4UBT,A2QAT"
983,144404,TMEM120B,uc001ubc.3,uc009zxh.2,2,20,0,107,0.02797965,A0PK00,None,Nothing,Nothing,1,1,1,0,0,1,1,1,1,"brca,coad,hnsc,kirc,luad,prad,thca",20,0.004502476,"A08BT,A03XT,A1IPT,A3X8T,A09TT,A5ZUT,A6JET,5100T,4799T,4841T,5044T,6361T,6498T,A2CRT,A41FT,A2Q1T,A39MT,A3ZQT,A3FOT,A13WT"
1061,149041,RC3H1,uc010pms.1,uc001gju.3,2,9,0,112,0.007330401,None,Q5TC82,Nothing,Nothing,0,1,1,0,0,1,1,1,0,"brca,coad,kich,kirc,prad",6,0.001350743,"A0E0T,6605T,8339T,4700T,5987T,6499T"
1122,151888,BTLA,uc003dza.3,uc003dzb.3,2,10,2,124,0.03788713,Q7Z6A9,Q7Z6A9-2,Nothing,Nothing,1,1,1,0,0,1,1,1,0,"brca,kirc,luad,lusc,prad,thca",18,0.004052229,"A204T,A2C9T,A158T,5703T,5591T,5677T,6742T,6831T,6972T,4626T,6982T,6829T,8082T,7125T,A3T8T,A1CST,A3PUT,A3ZTT"
1198,158381,ATP8B5P,uc010mkp.2,uc003zwu.2,2,8,0,82,0.01075012,None,None,Nothing,Nothing,0,1,0,0,0,0,0,0,1,"brca,hnsc,kirc,lihc",6,0.001350743,"A1AYT,A26GT,6827T,4641T,A112T,A1ECT"
1314,169436,C9orf96,uc004cdl.2,uc004cdk.2,2,9,0,129,0.005652621,None,Q8NE28,Nothing,Nothing,1,1,1,0,0,0,1,1,1,"brca,kirp,lihc,luad,lusc",13,0.00292661,"A56DT,A18GT,A3QQT,A0AWT,A08BT,A0E2T,A0STT,A5W8T,A114T,7815T,A5I6T,5485T,8250T"
1586,2199,FBLN2,uc011ava.1,uc011avb.1,21,10,27,33,0.03237577,P98095-2,P98095,HALLMARK_EPITHELIAL_MESENCHYMAL_TRANSITION,Nothing,1,1,1,0,0,1,1,1,0,"hnsc,luad,lusc",597,0.1343989,"7097T,7245T,6936T,6960T,7250T,7091T,7252T,8520T,8510T,8655T,A4JQT,7027T,6968T,4658T,8398T,8028T,6642T,8614T,6775T,4675T,A4SST,7765T,4507T,A4N5T,7910T,466